In [ ]:
import pandas as pd
import sqlite3
from pathlib import Path
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
db_path = Path('../../data/pipeline_agricultura.db').resolve()
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

conn = sqlite3.connect(db_path)
print(f"Conectado ao banco: {db_path}")

In [ ]:
query1 = '''
SELECT COUNT(*) as total_registros
FROM fato_producao
'''
resultado1 = pd.read_sql_query(query1, conn)
print("--- QUERY 1: Total de Registros na Tabela Fato ---")
print(f"Total de registros: {resultado1['total_registros'].iat[0]}")

In [ ]:
query2 = '''
SELECT
    dr.state_district AS regiao,
    SUM(fp.production) AS producao_total,
    AVG(fp.yield) AS rendimento_medio
FROM fato_producao fp
JOIN dim_regiao dr ON fp.id_regiao = dr.id_regiao
GROUP BY dr.state_district
ORDER BY producao_total DESC
LIMIT 10
'''
top_regioes = pd.read_sql_query(query2, conn)
print("--- QUERY 2: Top 10 Regioes por Producao Total (Usando Dimensao) ---")
print(top_regioes.to_string(index=False))

In [ ]:
query3 = '''
SELECT
    year,
    SUM(production) AS producao_total_anual,
    AVG(yield) AS rendimento_medio_anual
FROM fato_producao
GROUP BY year
ORDER BY year ASC
'''
producao_anual = pd.read_sql_query(query3, conn)
print("--- QUERY 3: Producao Total e Rendimento Medio por Ano ---")
print(producao_anual.to_string(index=False))

In [ ]:
query4 = '''
SELECT
    dc.crop AS cultura,
    COUNT(DISTINCT fp.id_regiao) AS total_regioes_produtoras,
    SUM(fp.production) AS producao_total,
    AVG(fp.yield) AS rendimento_medio_geral
FROM fato_producao fp
JOIN dim_cultura dc ON fp.id_cultura = dc.id_cultura
GROUP BY dc.crop
ORDER BY producao_total DESC
'''
segmento_cultura = pd.read_sql_query(query4, conn)
print("--- QUERY 4: Resumo de Desempenho por Cultura ---")
print(segmento_cultura.to_string(index=False))

In [ ]:
query5 = '''
SELECT
    state_district AS regiao,
    crop,
    coef_variacao_rendimento_percent AS cv_rendimento
FROM volatilidade_rendimento_regiao
WHERE crop = 'MAIZE' AND coef_variacao_rendimento_percent > 0
ORDER BY cv_rendimento ASC
LIMIT 5
'''
df_resultado5 = pd.read_sql_query(query5, conn)
print("--- QUERY 5: Top 5 Regioes Mais Estaveis para MAIZE ---")
print(df_resultado5.to_string(index=False))

In [ ]:
query6 = '''
SELECT
    state_district,
    rendimento_medio_local,
    rendimento_medio_nacional,
    diferenca_vs_media_percent
FROM benchmark_regional_rendimento
WHERE crop = 'RICE'
ORDER BY diferenca_vs_media_percent DESC
LIMIT 5
'''
df_resultado6 = pd.read_sql_query(query6, conn)
print("--- QUERY 6: Top 5 Regioes com Rendimento Acima da Media Nacional (RICE) ---")
print(df_resultado6.to_string(index=False))

In [ ]:
query7 = '''
SELECT
    year,
    rendimento_medio,
    crescimento_anual_rendimento_percent
FROM tendencia_anual_rendimento
WHERE crop = 'WHEAT'
ORDER BY year DESC
LIMIT 5
'''
df_resultado7 = pd.read_sql_query(query7, conn)
print("--- QUERY 7: Crescimento Anual do Rendimento para WHEAT (Ultimos 5 Anos) ---")
print(df_resultado7.to_string(index=False))

In [ ]:
query8 = '''
SELECT
    state_district,
    rendimento_medio,
    precipitacao_jun_sep_media,
    temp_media_junsep
FROM perfil_climatico_regiao_cultura
WHERE crop = 'RICE'
ORDER BY rendimento_medio DESC
LIMIT 1
'''
df_resultado8 = pd.read_sql_query(query8, conn)
print("--- QUERY 8: Perfil Climatico da Regiao Mais Produtiva (RICE) ---")
print(df_resultado8.to_string(index=False))

In [ ]:
query9 = '''
SELECT
    season,
    rendimento_medio,
    precipitacao_media_jun_sep,
    temp_media_jun_sep
FROM analise_sazonal_clima
WHERE crop = 'RICE'
ORDER BY rendimento_medio DESC
'''
df_resultado9 = pd.read_sql_query(query9, conn)
print("--- QUERY 9: Rendimento Medio de RICE por Temporada ---")
print(df_resultado9.to_string(index=False))

In [ ]:
query10 = '''
SELECT
    state_district AS regiao,
    crop,
    coef_variacao_rendimento_percent AS cv_rendimento,
    contagem_registros
FROM volatilidade_rendimento_regiao
WHERE crop = 'WHEAT'
ORDER BY cv_rendimento DESC
LIMIT 5
'''
df_resultado10 = pd.read_sql_query(query10, conn)
print("--- QUERY 10: Top 5 Regioes com Maior Volatilidade para WHEAT ---")
print(df_resultado10.to_string(index=False))

In [ ]:
query11 = '''
SELECT
    year,
    producao_total,
    rendimento_ponderado
FROM producao_anual_cultura
WHERE crop = 'MAIZE'
ORDER BY year DESC
'''
df_resultado11 = pd.read_sql_query(query11, conn)
print("--- QUERY 11: Producao Total e Rendimento Ponderado de MAIZE por Ano ---")
print(df_resultado11.to_string(index=False))

In [ ]:
query12 = '''
SELECT
    state_district,
    rendimento_medio_local,
    rendimento_medio_nacional,
    diferenca_vs_media_percent
FROM benchmark_regional_rendimento
WHERE crop = 'BARLEY'
ORDER BY diferenca_vs_media_percent ASC
LIMIT 5
'''
df_resultado12 = pd.read_sql_query(query12, conn)
print("--- QUERY 12: Top 5 Regioes com Rendimento Abaixo da Media Nacional (BARLEY) ---")
print(df_resultado12.to_string(index=False))

In [ ]:
conn.close()
print('Conexao encerrada.')